# Задание 3.

Загрузите [данные по изменению температуры поверхности земли](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data). Для этого может понадобится зарегистрироваться на [Kaggle](https://kaggle.com). Затем нужно будет работать с данными, которые содержатся в файле **GlobalLandTemperaturesByMajorCity.csv**

In [ ]:
import zipfile

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

with zipfile.ZipFile("archive.zip") as z:
    with z.open("GlobalLandTemperaturesByMajorCity.csv") as f:
        df = pd.read_csv(f, parse_dates=["dt"])

df.head()

# Задание 3.1 (1 балл)

В последующих заданиях будут учитываться данные начиная с 01.01.1950. Для этого создайте новый `DataFrame`, в котором удалены все строки до 01.01.1950. Используйте созданный DataFrame в последующих заданиях.  

# Задание 3.2 (2 балла)

Постройте график среднегодовых температур в городе Торонто.

# Задание 3.3 (2 балла)

Найдите город, для которого выборочная дисперсия температур на приведенных данных максимальна. 

# Задание 3.4 (3 балла)

На одной оси постройте графики, отображающие:
1. Зависимость выборочной дисперсии температур от широты города (Latitude).
2. Зависимость средней температуры в январе 1975 года от широты города (Latitude).
3. Зависимость средней температуры в июле 1975 года от широты города (Latitude).

# Задание 3.5 (4 балла)

Посчитайте данные по среднегодовой температуре в Санкт-Петербурге. Определите года, в которых средняя температура была выше, чем в предыдущем  и последующем году. Постройте график среднегодовой температуры и обозначьте на нём найденные года (произвольным образом). 

Постройте модель линейной регрессии (можно использовать [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)): $\mathbb{год} \rightarrow \mathbb{средняя\_температура}$. Отобразите прямую на предыдущем графике. 

# Задание 3.6 (3 балла)

Найдите города, для которых: 
1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.
2. Самая большая средняя разница между средней температурой января и средней температурой июля.
3. Наибольшее среднее количество месяцев с отрицательной температурой в году.

# Задание 3.7 (3 балла)

Отобразите города на карте, например с помощью [geopandas/folium](https://geopandas.org/en/stable/gallery/plotting_with_folium.html) или [plotly](https://plotly.com/python-api-reference/generated/plotly.express.scatter_geo.html). Причем цвет метки должен соответствовать средней температуре в январе за последние 10 лет. 


**\***(+2 балла). Вычислите прямую с "трендом" по изменению средней температуры за год в каждом городе в период 1950-2013 (можно с помощью линейной регрессии, как в задании **3.5**). Отобразите города цветом, который зависит от угла наклона этой прямой. 


**\***(+1 балл). В дополнение к предыдущему пункту, используйте библиотеку [pwlf](https://pypi.org/project/pwlf/) с фиксированным числом сегментов и визуализируйте угол наклона последнего сегмента. 

# Задание 3.8 (5 баллов)



Метод [`rolling`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html#pandas.DataFrame.rolling) поддерживает различные виды окон - по временному интервалу, абсолютному количеству элементов и окно с произвольной логикой, через переопределение класса [`BaseIndexer`](https://pandas.pydata.org/docs/reference/api/pandas.api.indexers.BaseIndexer.html#pandas.api.indexers.BaseIndexer)

Создайте класс `TimeOffsetBasedIndexer`, который задаёт кастомное временное окно. Окно должно учитывать временные метки (в индексе или произвольном столбце) и предусматривать 
отступ (`offset`) правой границы влево от текущей позиции. 

То есть требуется реализовать класс `TimeOffsetBasedIndexer`, унаследованный от `BaseIndexer`.
Параметры класса:
  - `window`: длина временного окна (`pd.Timedelta`).
  - `offset`: величина отступа влево от текущей позиции (`pd.Timedelta`).

Метод [`get_window_bounds()`](https://pandas.pydata.org/pandas-docs/version/1.0.0/reference/api/pandas.api.indexers.BaseIndexer.get_window_bounds.html) должен определять границы окна (с учетом `window` и `offset`), при этом другие параметры (`center`, `closed`, и т.д.) можно игнорировать.

Пример использования:
```python
df = pd.read_parquet("../data/timeseries.snappy.parquet")

indexer = TimeBasedIndexer(window=pd.Timedelta("25s"), offset=pd.Timedelta("5s"))
result = df.rolling(indexer, on="timestamp", min_periods=1).mean()
```

В данном случае считаются средние значения по окнам ширины 25 секунд со смещением (запаздыванием) влево на 5 секунд.

Для каждой точки $x$ в `df` граница окна определяется следующим образом:
1. Вычисляется значение времени $x - offset$. Правая граница окна имеет timestamp строго меньше $x - offset$.
2. Вычисляется значение времени $x - offset - window$. Левая граница окна имеет время больше или равное  $x - offset - window$.

**\***(+3 балла). Измените правило определения левой границы окна. Если $x - offset - window$ не попадает в существующую точку в df, то граница сдвигается влево. Если слева 
серия серия точек с одинаковым значением временной метки, то берется самая левая такая точка. Используйте numba для оптимизации производительности.